# Transformación de datos

La transformación de datos es el proceso en el que se cambia el formato o estructura de un dataset para poder ser usado o procesado. Otras formas de referirse a este proceso puede ser **data wrangling** o **data munging**.

El **data wrangling** es el proceso de transformar los datos de un *raw dataset* (en bruto) u otro formato, con la intención de hacerlo más apropiado y valioso para una variedad de propósitos posteriores, como puede ser su análisis o carga en un sistema determinado.

Este proceso puede incluir visualización de datos, agregación de datos, entrenamiento de un modelo estadístico, así como muchos otros usos potenciales. La oscilación de datos como proceso generalmente sigue un conjunto de pasos generales que comienzan extrayendo los datos en forma cruda del origen de datos, dividiendo los datos en bruto usando algoritmos (por ejemplo, clasificación) o analizando los datos en estructuras de datos predefinidas, y finalmente depositando el contenido resultante en un sistema de almacenamiento (datawarehouse) para su uso futuro, se puede denominar también a esto un proceso ETL (Extract - Transform - Load).

In [2]:
## Importamos librerias
import pandas as pd
import numpy as np

In [3]:
## Cargamos ficheros de trabajo
mainpath = "C:/Users/gmachin/Desktop/Developer/apuntes-notebooks/datasets"
filename = "customer-churn-model/customer-churn-model.txt"
fullpath = mainpath + "/" + filename

df_churn = pd.read_csv(fullpath)

df_churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


## 1. Copiar un DataFrame

Una buena práctica es hacer una copia de todos los dataframes nada más cargarlos, de modo que en cualquier momento del proceso podamos comparar nuestro dataset de trabajo con el original, para ver cómo va variando.

Para ello utilizamos la función `df.copy()`. En el argumento de la función deberemos indicar el argumento `deep = True`, de este modo aseguramos que lo que realizamos es una copia auténtica del fichero; si en este parámetro indicásemos `deep = False`, lo que estaríamos haciendo sería crear un vínculo entre ambos ficheros y cualquier modificación que realizaramos en alguno de los dos, se aplicaría en el otro, como un espejo.

In [7]:
df_churn_copy = df_churn.copy(deep = True)

## 2. Eliminar y añadir información en un DataFrame

### 2.1. Eliminar columnas y filas

Podemos eliminar columnas de un DataFrame mediante la función `df.drop(columnas, axis = 1)`. En caso de ser muchas columnas podemos almacenarlas en una variable, o al contrario, realizar una extracción de las columnas de interés eliminando por ende el resto.

In [16]:
df_churn.drop(['Account Length', 'Phone', 'Churn?', 'CustServ Calls'], axis = 1).head(2)

,State,Area Code,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge
0,KS,415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.7
1,OH,415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7


Para eliminar las filas podermos valernos de la misma función pero indicando el índice de estas, y cambiando el argumento axis a 0. `df.drop(indice, axis = 0)`.

In [19]:
df_churn.drop([1, 2 , 3], axis = 0).head(5)

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.
5,AL,118,510,391-8027,yes,no,0,223.4,98,37.98,...,101,18.75,203.9,118,9.18,6.3,6,1.70,0,False.
6,MA,121,510,355-9993,no,yes,24,218.2,88,37.09,...,108,29.62,212.6,118,9.57,7.5,7,2.03,3,False.
7,MO,147,415,329-9001,yes,no,0,157.0,79,26.69,...,94,8.76,211.8,96,9.53,7.1,6,1.92,0,False.


### 2.2. Añadir columnas

Podemos crear columnas nuevas con valores fijos indicando la tabla y el valor fijo con el que queremos completar la nueva columna. La sintaxis sería `df[columna_nueva] = valor`.

Este valor puede ser un número entero, un string, una fecha...etc.

In [3]:
## Vamos a crear un dataset mas pequeño para este ejercicio, para que sea más cómodo de trabajar:
columnas = ['Day Calls', 'Eve Calls', 'Night Calls']
df_churn_nuevo = df_churn[columnas]

df_churn_nuevo.head(3)

,Day Calls,Eve Calls,Night Calls
0,110,99,91
1,123,103,103
2,114,110,104


In [24]:
## Añadimos una columna nueva que contenga el string "suman":
df_churn_nuevo['suman'] = 'suman'

df_churn_nuevo.head(3)

C:\Users\gmachin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Day Calls,Eve Calls,Night Calls,suman
0,110,99,91,suman
1,123,103,103,suman
2,114,110,104,suman


También se pueden añadir columnas nuevas a partir de la información del resto de columnas. La sintaxis para llevar a cabo esta tarea sería definir la columna que queremos crear, e igualarla a la operación matemática o de strings que queramos.

`df[columna_nueva] = df[columna_1] + df[columna_2] * df[columna_3]`

`df[columna_nueva] = df[columna_4] + "Hola Mundo!"`

In [27]:
## Creamos una columna que sume el número total de columnas en el día
df_churn_nuevo['total calls'] = df_churn_nuevo['Day Calls'] + df_churn_nuevo['Eve Calls'] + df_churn_nuevo['Night Calls']
        
df_churn_nuevo.head(3)

C:\Users\gmachin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Day Calls,Eve Calls,Night Calls,suman,total calls
0,110,99,91,suman,300
1,123,103,103,suman,329
2,114,110,104,suman,328


A las prácticas anteriores de crear columnas, se le pueden añadir condicionales de modo que se realicen ciertas operaciones en función de que se cumplan unas condiciones prefijadas o no. Como Python es un lenguaje muy versatil, se puede realizar de diferentes formas, a continuación analizaremos algunas de ellas.

**Mediante diccionarios:**

In [28]:
## Creamos una tabla de muestra.
df = pd.DataFrame({'Id': ['A', 'B', 'C', 'D'],
                   'Value': [10, 9, 8, 7]})

## Definimos el diccionario que contiene los pares de datos, y mapeamos los valores
diccionario = {'A': 1, 'B': 2, 'C': 3, 'D': 4} 
df['Value_2'] = df['Id'].map(diccionario)

df

,Id,Value,Value_2
0,A,10,1
1,B,9,2
2,C,8,3
3,D,7,4


**Mediante una función que contenga los condicionales definidos. Este metodo es más lento, pero permite jugar con los valores de diferentes columnas:**

In [29]:
## Creamos el dataframe:
df = pd.DataFrame({'Type': ['A', 'B', 'B', 'C'],
                   'Set': ['Z', 'Z', 'X', 'Y']})

## Definimos la función con sus condicionales:
def set_color(row):
    if row["Set"] == "Z":
        return "red"
    elif row["Type"] == "C":
        return "blue"
    else:
        return "green"

## Creamos la columna nueva con los resultados de la función:
df = df.assign(color = df.apply(set_color, axis=1))
df

,Type,Set,color
0,A,Z,red
1,B,Z,red
2,B,X,green
3,C,Y,blue


**Mediante la función select de la librería NumPy:**

In [30]:
## Creamos el dataframe:
df = pd.DataFrame({'Type': ['A', 'B', 'B', 'C'],
                   'Set': ['Z', 'Z', 'X', 'Y']})

## Definimos los diferentes condicionales:
conditions = [
    (df['Set'] == 'Z') & (df['Type'] == 'A'),
    (df['Set'] == 'Z') & (df['Type'] == 'B'),
    (df['Type'] == 'B')]
choices = ['yellow', 'blue', 'purple']

## Mapeamos
df['color'] = np.select(conditions, choices, default='black')
df

,Type,Set,color
0,A,Z,yellow
1,B,Z,blue
2,B,X,purple
3,C,Y,black


## 2.3. Eliminar duplicados

Se pueden eliminar los duplicados de un DataFrame atiendo a unos campos llave, mediante el método `df.drop_duplicates(subset, opt)`, donde *subset* hace referencia a las columnas que se van a emplear para identificar los duplicados, y *opt* diferentes argumentos que permitirán que esta función trabaje de un modo u otro.

In [4]:
df = pd.DataFrame({'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
                   'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
                   'rating': [4, 4, 3.5, 15, 5]})
df

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


In [5]:
df.drop_duplicates(subset=['brand', 'style'], keep='last')

,brand,style,rating
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
4,Indomie,pack,5.0


### 2.4. Dumificar variables

De forma habitual trabajaremos con datos que contienen información en variables categóricas, continuas, discretas...etc, esto puede marcar significativamente el modelo de machine learning que terminemos utilizando. No obstante, existe una práctica denominada dumificación de variables, que consiste en convertir una variable categórica con $n$ posible valores, a $n$ columnas con valores dicotómicos.

Esta dumificación, puede permitir por ejemplo incuir variables cualitativas en modelos de regresión lineal, los cuales en principio están acotados a datos de tipo cuantitativo.

Para dumificar una variable se utiliza la función `pd.get_dummies(df['column'], prefix)`:

In [6]:
## Creamos un DataFrame
diccionario = {'id': ['Juan', 'Luis', 'Ana', 'Raquel'],
               'genero': ['masculino', 'masculino', 'femenino', 'femenino']}
df = pd.DataFrame(diccionario)

## Dumificamos el género
df_dummy = pd.get_dummies(df["genero"], prefix = "genero")

df_dummy

,genero_femenino,genero_masculino
0,0,1
1,0,1
2,1,0
3,1,0


## 3. Crear subconjuntos de datos

Es posible que muchas veces, de todo el DataFrame con el que vamos a trabajar, nos interesen solo algunas columnas. También puede ocurrir que para un análisis en concreto queramos utilizar tablas auxiliares creando un subconjunto de datos del DataFrame principal.

### 3.1. Subconjunto de columnas

La sintaxis a seguir a la hora de extraer un subconjunto de columnas de un dataframe, es la siguiente:

`df_destino = df_origen[["nombre_columna_1", "nombre_columna_2", [...], [...], "nombre_columna_n"]]`

También se puede almacenar todas las columnas que queramos en una lista, y pasarla como argumento en la estructura anterior.

In [35]:
columnas = ['State', 'Area Code', 'Day Mins', 'Day Calls', 'Night Mins', 'Night Calls']
df_churn_new = df_churn[columnas]

df_churn_new.head(3)

,State,Area Code,Day Mins,Day Calls,Night Mins,Night Calls
0,KS,415,265.1,110,244.7,91
1,OH,415,161.6,123,254.4,103
2,NJ,415,243.4,114,162.6,104


### 3.2. Subconjunto de filas

Al igual que se ha en el punto anterior, también se pueden crear subconjuntos de datos filtrando por determinadas filas. En este caso, lo haremos en base al índice de los registros.

**Nota: es importante tener en cuenta que en Python, el primer registro tiene índice 0.**

La sintaxis a seguir es: `df_destino = df_origen [fila_inicio:fila_fin]`

Existen dos casos particulares, que permiten elegir desde el inicio del fichero hasta el registro `x`, o desde el registro `X` hasta el final del mismo:

- `df_destino = df_origen[:x]`
- `df_destino = df_origen[x:]`

In [39]:
df_churn_new2 = df_churn_new[:5]
df_churn_new2

,State,Area Code,Day Mins,Day Calls,Night Mins,Night Calls
0,KS,415,265.1,110,244.7,91
1,OH,415,161.6,123,254.4,103
2,NJ,415,243.4,114,162.6,104
3,OH,408,299.4,71,196.9,89
4,OK,415,166.7,113,186.9,121


### 3.3. Subconjuntos de filas y columnas en base a condiciones booleanas

En los DataFrames de Pandas existen múltiples formas de seleccionar los registros de las filas y columnas, siendo dos de las más importantes `iloc` y `loc`. La primera permite seleccionar los elementos en base a la posición de las filas y columnas dentro del DataFrame (utilizando índices), mientras que la segunda permite seleccionar mediante etiquetas.

- `iloc`: en base a la posición. Sintaxis: `df.iloc[filas, columnas]`.
- `loc`: en base a etiquetas. Sintaxis: `df.loc[filas, columnas]`.

Para incluir los condicionales se deben pasar como argumentos de la función en la parte de las filas. Hay que tener en cuenta que para concatenar operadores lógicos en pandas se debe utilizar el símbolo `&` para el operador `and`; y el símbolo `|` para el operador `or`.

- `&` = `and`
- `|` = `or`

Ejemplo de la sintaxis:

`df.loc[df[columna_1] > x & df[columna_1] < y , [columan_1, columna_2]]`

*¡Nota!: con el método `.iloc` solo se coge el inicio pero no el fin, por eso al hacer `data.iloc[0:5]` selecciona desde la fila 0 hasta la 4, a diferencia del método `.loc` `data.loc[0:5]` el cual coge desde la fila 0 hasta la fila 5.*

In [5]:
## Creamos un DataFrame para realizar los ejercicios
columnas = ['State', 'Area Code', 'Day Mins', 'Day Calls', 'Night Mins', 'Night Calls']
df_churn_new = df_churn[columnas]

df_churn_new.head(3)

,State,Area Code,Day Mins,Day Calls,Night Mins,Night Calls
0,KS,415,265.1,110,244.7,91
1,OH,415,161.6,123,254.4,103
2,NJ,415,243.4,114,162.6,104


In [47]:
## Seleccionar todas las columnas desde el principio hasta la fila 10
df_churn_new.loc[:10, :]

,State,Area Code,Day Mins,Day Calls,Night Mins,Night Calls
0,KS,415,265.1,110,244.7,91
1,OH,415,161.6,123,254.4,103
2,NJ,415,243.4,114,162.6,104
3,OH,408,299.4,71,196.9,89
4,OK,415,166.7,113,186.9,121
5,AL,510,223.4,98,203.9,118
6,MA,510,218.2,88,212.6,118
7,MO,415,157.0,79,211.8,96
8,LA,408,184.5,97,215.8,90
9,WV,415,258.6,84,326.4,97


In [48]:
## Seleccionar las dos primeras columnas de las primeras 5 filas
df_churn_new.loc[:5, ['State', 'Area Code']]

,State,Area Code
0,KS,415
1,OH,415
2,NJ,415
3,OH,408
4,OK,415
5,AL,510


In [6]:
## Seleccionar las columnas State y Day Calls de aquellos registros que tengan más de 100 llamadas al día y
## que sean del estado de "KS" o "OH". Mostrar los 5 primeros resultados.

df_churn_new.loc[(df_churn_new["Day Calls"]>100) & ((df_churn_new["State"]=="KS") | (df_churn_new["State"]=="OH")),
                 ["State", "Day Calls"]][:5]

,State,Day Calls
0,KS,110
1,OH,123
152,OH,116
156,OH,120
179,KS,122


También podemos emplear métodos más sencillos si solo queremos filtrar filas en base a condicionales y queremos todas las columnas del DataFrame.

In [54]:
df_churn_new[df_churn_new['State'].isin(['AL', 'MO'])][:5]

,State,Area Code,Day Mins,Day Calls,Night Mins,Night Calls
5,AL,510,223.4,98,203.9,118
7,MO,415,157.0,79,211.8,96
28,MO,415,190.0,109,181.5,102
112,AL,408,161.0,117,227.7,113
118,MO,510,113.7,117,177.6,118


In [16]:
df_churn_new[df_churn_new['Day Mins'] > 300][:5]

,State,Area Code,Day Mins,Day Calls,Night Mins,Night Calls
15,NY,415,332.9,67,160.6,128
76,DC,415,300.3,109,270.1,73
156,OH,415,337.4,120,153.9,114
197,TX,510,326.5,67,181.7,102
230,MD,408,312.0,109,217.6,74


## 4. Agregaciones

Otra de las operaciones más comunes que se pueden encontrar dentro del ámbito del Data Wrangling es la agrupación de los datos en base a alguna/s de las columnas del dataset.

Para llevar a cabo esta operación se utiliza la función `df.groupby(['columna_1', 'columna_2'])`. Detrás de esta sintaxis, se pueden añadir otras funciones para sumarizar los valores, obtener la media, o contar el número de filas del resto de variables.

In [11]:
## Calcular el número de minutos totales hablados y llamadas por estado. 5 primero resultados:
df_churn_new[['State', 'Day Mins', 'Day Calls']].groupby(["State"]).sum().head()

,Day Mins,Day Calls
State,,
AK,9276.0,4954
AL,14880.8,7928
AR,9686.4,5618
AZ,10982.7,6407
CA,6241.2,3553


In [13]:
## Calcular el número total de minutos de día, y la media de llamadas diarias. 5 primero resultados:
df_churn_new.groupby(["State"]).agg({"Day Mins": "sum",
                                     "Day Calls": "mean",
                                    }).reset_index().round(2).head()

,State,Day Mins,Day Calls
0,AK,9276.0,95.27
1,AL,14880.8,99.10
2,AR,9686.4,102.15
3,AZ,10982.7,100.11
4,CA,6241.2,104.50


## 5. Combinación de DataFrames

En muchas ocasiones nos podemos encontrar con que los conjuntos de datos no se encuentran agregados en una única tabla, ya sea porque por ejemplo trabajamos con series temporales y los datos se guardan cada día en un fichero distinto, o porque la BBDD con la que trabajamos distribuye la información en diferentes tablas.

### 5.1. Concatenar

Concatenar es el proceso en el que se junta información que tiene el mismo formato, ya sea en sentido horizontal o vertical:

- Concatenar vertical: Ficheros con el mismo número de columnas y coincidentes en el orden de éstas. Concatenar vertical coloca las información del segundo fichero debajo del primero, aumentando el número de filas pero manteniéndose constante el número de columnas.

- Concatenar horizontal: Ficheros con el mismo número de filas, y que el index de cada fichero es coincidente. Concatenar horizontal coloca la información del segundo fichero a la derecha del primero, aumentando el número de columnas pero manteniéndose constante el número de filas.

Para realizar este proceso en Python utilizaremos la función `df_new = pd.concat([df1, df2, ..., dfn], axis)`. Debemos indicar el sentido de la concatenación mediante el argumento `axis`.

- `axis = 0` para el concatenamiento vertical
- `axis = 1` para el concatenamiento horizontal

Para realizar el ejemplo de este ejercicio cargamos unos nuevos datasets:

In [9]:
## Cargamos ficheros de trabajo
mainpath = "C:/Users/gmachin/Desktop/Developer/apuntes-notebooks/datasets"

filename = "wine/winequality-white.csv"
fullpath = mainpath + "/" + filename
df_ww = pd.read_csv(fullpath, sep = ";")

filename = "wine/winequality-red.csv"
fullpath = mainpath + "/" + filename
df_rw = pd.read_csv(fullpath, sep = ";")

df_ww.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6


In [15]:
## Comprabamos las dimensiones de ambos ficheros, deben ser mismo número de columnas
print(df_ww.shape)
print(df_rw.shape)

(4898, 12)
(1599, 12)


In [12]:
## Concatenamos los ficheros verticalmente, uno debajo del otro
wine_data = pd.concat([df_ww, df_rw], axis = 0)
wine_data.shape

(6497, 12)

Podemos complementar la funcion `pd.concat()` con un bucle para cargar multitud de ficheros distribuidos, siempre y cuando sigan algún tipo de regla lógica de numeración.

Un ejemplo podría ser un dataset de estaciones meteorológicas, en el que la información de cada estación viene en un fichero diferente:

- Estación 1: est_met_1.csv
- Estación 2: est_met_2.csv
- ...
- Estación n: est_met_n.csv

A continuación se muestra un ejemplo de como se podría agrupar todos estos ficheros de forma sencilla:

In [ ]:
## Definimos la ruta donde se ecuentran todos los ficheros a concatenar
filepath = "C:/Users/gmachin/Desktop/Developer/apuntes-notebooks/datasets/distributed-data/"

## Cargamos el primer fichero
data = pd.read_csv(fullpath)
final_length = len(data)

## Ya hemos cargado 1/332 ficheros, por lo que comenzamos el bucle en el segundo. (No entra el último en el range)
## En el rango obtenemos los datos en formato 1, 2, 3...etc y nosotros queremos ponerlo en formato 001, 002, 003...etc
for i in range(2,333):
    if i < 10:
        filename = "00" + str(i)
    if 10 <= i < 100:
        filename = "0" + str(i)
    if i >= 100:
        filename = str(i)
        
    ## Definimos el nombre del .csv utilizando los valores que se han generado en el bucle
    file = filepath + filename + ".csv"
    temp_data = pd.read_csv(file)
    
    ## Concatenamos cada tabla
    data = pd.concat([data, temp_data], axis = 0)

### 5.2. Joins

Los Joins sirven para combinar filas de dos o más tablas basándose en un campo común entre ellas, devolviendo por tanto datos de diferentes tablas, están basados en los Joins de SQL. Para llevar a cabo esta operación en Python se utiliza la función `pd.merge(df1, df2, how, left_on, right_on, suffixes)`.

Para especificar el tipo de join se debe indicar mediante el argumento `how`.

- `how = 'left'`: Left Join
- `how = 'right'`: Right Join
- `how = 'inner'`: Inner Join
- `how = 'outer'`: Full-Outer Join

Mediante los argumentos `left_on` y `right_on` se indican los campos que actúan como llaves entre ambos DataFrames.

Adicionalmente podemos completar tambien el argumento `suffixes` para identificar aquellas columnas con nombres iguales de que tabla proceden. Un ejemplo podría ser `suffixes=('_left', '_right')`.

Otro punto a mencionar de la funcion `pd.merge()` de `pandas` es que por defecto es del tipo 1 $\rightarrow$ n. Es decir, si para un registro de la primera tabla, existen 2 registros en la segunda, el fichero de salida tendrá dos registros para este caso.

Para llevar a cabo esta parte cargamos unos ficheros nuevos:

In [14]:
## Cargamos ficheros de trabajo
mainpath = "C:/Users/gmachin/Desktop/Developer/apuntes-notebooks/datasets/clientes-carteras"

filename = "clientes.xlsx"
fullpath = mainpath + "/" + filename
df_clientes = pd.read_excel(fullpath)

filename = "carteras.xlsx"
fullpath = mainpath + "/" + filename
df_carteras = pd.read_excel(fullpath)

filename = "gestores.xlsx"
fullpath = mainpath + "/" + filename
df_gestores = pd.read_excel(fullpath)

In [15]:
df_clientes

,id_cliente,des_cliente
0,1,Juan Juánez
1,2,Luis Luisez
2,3,Diego Diéguez
3,4,Ana García
4,5,Isabel Preysler
5,6,Ana de Armas
6,7,Belen Estébanez
7,8,Roberto Carlos Jr.
8,9,Florentino Fernández
9,10,David Broncano


In [16]:
df_carteras

,id_cliente,id_cartera,des_cartera
0,1,C0001,Cartera 1
1,2,C0001,Cartera 1
2,3,C0001,Cartera 1
3,4,C0001,Cartera 1
4,5,C0001,Cartera 1
5,6,C0002,Cartera 2
6,7,C0003,Cartera 3
7,8,C0004,Cartera 4
8,9,C0004,Cartera 4
9,11,C0010,Cartera 10


In [17]:
df_gestores

,id_cartera,id_gestor,des_gestor
0,C0001,G0001,Ana Botín
1,C0002,G0001,Ana Botín
2,C0003,G0001,Ana Botín
3,C0003,G0002,Andrea Orcel
4,C0004,G0002,Andrea Orcel


#### 5.2.1. Left Join

Este tipo de Join muestra todos los resultados de la primera tabla A (o de la izquierda), y en caso de haber match el valor de los registros de la segunda tabla B, si no existe correspondencia obtendremos como resultado un valor `NaN`.

In [20]:
##Analizamos si los clientes tiene una cartera asignada o no:

pd_left_join = pd.merge(df_clientes,
                        df_carteras,
                        how = 'left',
                        left_on = 'id_cliente',
                        right_on = 'id_cliente')

pd_left_join

,id_cliente,des_cliente,id_cartera,des_cartera
0,1,Juan Juánez,C0001,Cartera 1
1,2,Luis Luisez,C0001,Cartera 1
2,3,Diego Diéguez,C0001,Cartera 1
3,4,Ana García,C0001,Cartera 1
4,5,Isabel Preysler,C0001,Cartera 1
5,6,Ana de Armas,C0002,Cartera 2
6,7,Belen Estébanez,C0003,Cartera 3
7,8,Roberto Carlos Jr.,C0004,Cartera 4
8,9,Florentino Fernández,C0004,Cartera 4
9,10,David Broncano,NaN,NaN


Vemos que todos tienen una cartera asignada menos David Broncano, a pesar de no tener cartera, como se ha realizado un left join aparece el cliente pero con el valor de la cartera `NaN`.

#### 5.2.2. Right Join

Es la misma tipología del Left Join, solo que la primera tabla en este caso es la de la derecha. Como buena práctica se suele establecer el uso único del Left Join, de modo que la estructura siempre similar, y facilitar el entendimiento del código.

#### 5.3.3. Inner Join

Este tipo de Join devuelve sólo aquellas filas donde haya un match, es decir, las filas donde el valor del campo de la tabla A que se utiliza para hacer el Join coincida con el valor del campo correspondiente en la tabla B.

In [9]:
pd_inner_join = pd.merge(df_clientes,
                         df_carteras,
                         how = 'inner',
                         left_on = 'id_cliente',
                         right_on = 'id_cliente')

pd_inner_join

,id_cliente,des_cliente,id_cartera,des_cartera
0,1,Juan Juánez,C0001,Cartera 1
1,2,Luis Luisez,C0001,Cartera 1
2,3,Diego Diéguez,C0001,Cartera 1
3,4,Ana García,C0001,Cartera 1
4,5,Isabel Preysler,C0001,Cartera 1
5,6,Ana de Armas,C0002,Cartera 2
6,7,Belen Estébanez,C0003,Cartera 3
7,8,Roberto Carlos Jr.,C0004,Cartera 4
8,9,Florentino Fernández,C0004,Cartera 4


Como vimos en el punto anterior, David Broncano no tiene una cartera asignada, por lo que en este caso no aparece en el resultado. Si quisieramos quedarnos solo con auqellos registros que no han cruzado, deberíamos realizar un Left Join, y después añadir otra query para quedarnos solo con aquellos que tienen valores `NaN`.

Por otro lado, podemos comprobar que este tipo de Joins por defecto es del tipo 1 $\rightarrow$ n. Es decir, si para un registro de la primera tabla, existen 2 registros en la segunda, el fichero de salida tendrá dos registros para este caso.

Podemos comprobarlo en nuestros ficheros para el caso de Belen Estébanez, la cual tiene dos gestores de cartera.

In [10]:
pd_inner_join2 = pd.merge(pd_left_join,
                         df_gestores,
                         how = 'inner',
                         left_on = 'id_cartera',
                         right_on = 'id_cartera')

pd_inner_join2

,id_cliente,des_cliente,id_cartera,des_cartera,id_gestor,des_gestor
0,1,Juan Juánez,C0001,Cartera 1,G0001,Ana Botín
1,2,Luis Luisez,C0001,Cartera 1,G0001,Ana Botín
2,3,Diego Diéguez,C0001,Cartera 1,G0001,Ana Botín
3,4,Ana García,C0001,Cartera 1,G0001,Ana Botín
4,5,Isabel Preysler,C0001,Cartera 1,G0001,Ana Botín
5,6,Ana de Armas,C0002,Cartera 2,G0001,Ana Botín
6,7,Belen Estébanez,C0003,Cartera 3,G0001,Ana Botín
7,7,Belen Estébanez,C0003,Cartera 3,G0002,Andrea Orcel
8,8,Roberto Carlos Jr.,C0004,Cartera 4,G0002,Andrea Orcel
9,9,Florentino Fernández,C0004,Cartera 4,G0002,Andrea Orcel


#### 5.3.4. Full Outer Join

Este Join permite combinar los datos de dos tablas mostrando tanto los datos que cumplen la llave de cruce como los que no, éstos últimos apareceran con el valor `NaN`.

In [13]:
pd_outer_join = pd.merge(df_clientes,
                         df_carteras,
                         how = 'outer',
                         left_on = 'id_cliente',
                         right_on = 'id_cliente')

pd_outer_join

,id_cliente,des_cliente,id_cartera,des_cartera
0,1,Juan Juánez,C0001,Cartera 1
1,2,Luis Luisez,C0001,Cartera 1
2,3,Diego Diéguez,C0001,Cartera 1
3,4,Ana García,C0001,Cartera 1
4,5,Isabel Preysler,C0001,Cartera 1
5,6,Ana de Armas,C0002,Cartera 2
6,7,Belen Estébanez,C0003,Cartera 3
7,8,Roberto Carlos Jr.,C0004,Cartera 4
8,9,Florentino Fernández,C0004,Cartera 4
9,10,David Broncano,NaN,NaN
